In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-chunks/gnostic_chunks.json
/kaggle/input/text-chunks/bible_chunks.json


In [109]:
import json

with open("/kaggle/input/text-chunks/bible_chunks.json","r") as f:
    bible = json.load(f)

with open("/kaggle/input/text-chunks/gnostic_chunks.json","r") as f:
    gnostic = json.load(f)

In [4]:
mat = bible['Matthew']
got = gnostic['Gospel of Thomas']

In [3]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer("all-MiniLM-L6-v2",device='cuda')
def embed(text):
    embedding = model.encode(text,show_progress=True, convert_to_numpy=True)
    return embedding

2025-04-20 06:26:33.960260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745130394.148889      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745130394.205221      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
b = []
for k in bible.keys():
    b.extend(bible[k])

In [7]:
g = []
for k in gnostic.keys():
    g.extend(gnostic[k])

In [9]:
bemb = embed(b)
gemb = embed(g)

Batches:   0%|          | 0/198 [00:00<?, ?it/s]

Batches:   0%|          | 0/262 [00:00<?, ?it/s]

In [11]:
cos = util.cos_sim(gemb,bemb)

In [13]:
print(gemb.shape,bemb.shape,cos.shape)

(8373, 384) (6307, 384) torch.Size([8373, 6307])


In [24]:
ri,ci = np.where(cos>0.5)

In [27]:
pairs = []
for r,c in zip(ri,ci):
    score = cos[r,c]
    gt = g[r]
    bt = b[c]
    pairs.append((gt,bt,score.item()))

In [6]:
import spacy
import matplotlib.pyplot as plt
import re

nlp = spacy.load("en_core_web_sm")

def extract_entities_and_relations(paragraphs):
    """
    Extract cosmological entities and relationships using NER and dependency parsing.
    Args:
        paragraphs (list): List of text paragraphs.
    Returns:
        tuple: Lists of entities and relationships.
    """
    entities = set()
    relationships = []
    
    for para in paragraphs:
        doc = nlp(para)
        # Extract entities (customize for Gnostic terms)
        for ent in doc.ents:
            if ent.label_ in ["DIVINE", "NORP", "REALM"]:  # Adjust for divine beings, realms
                entities.add(ent.text)
        
        # Extract relationships (simplified example)
        for sent in doc.sents:
            for token in sent:
                if token.lemma_ in ["emanate", "beget", "create", "produce"]:
                    # Look for subject and object
                    subject = None
                    obj = None
                    for child in token.children:
                        if child.dep_ in ["nsubj", "nsubjpass"]:
                            subject = child.text
                        if child.dep_ in ["dobj", "attr"]:
                            obj = child.text
                    if subject and obj:
                        relationships.append((subject, obj, token.lemma_))
    
    return list(entities), relationships

In [38]:
ow = gnostic['On the Origin of the World']

In [8]:
en,rl = extract_entities_and_relations(g)

In [14]:
doc= nlp("".join(g))

In [17]:
nc = [chunks.text for chunks in doc.noun_chunks]

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
vect = TfidfVectorizer(ngram_range = (1,3),stop_words='english')
x=vect.fit_transform(nc)

In [79]:
fn=vect.get_feature_names_out()

In [80]:
len(fn)

7923

In [81]:
scores = x.sum(axis=0).A1

In [85]:
top=scores.argsort()[-20:]

In [86]:
tp = [fn[i] for i in top]

In [87]:
tp

['soul',
 'order',
 'powers',
 'glory',
 'word',
 'lines',
 'truth',
 'aeons',
 'life',
 'spirit',
 'knowledge',
 'world',
 'son',
 'place',
 'power',
 'man',
 'light',
 'god',
 'things',
 'father']

## Entity extraction bookwise

In [110]:
from collections import defaultdict

In [ ]:
!python -m spacy download en_core_web_lg

In [116]:
nlp = spacy.load("en_core_web_lg")

In [136]:
def clean_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[.!?]+$', '.', text)
    text = re.sub(r'[!@#%^&*()_+={}\[\]|\\:;"\'<>,?/~`]+', '', text)
    text = re.sub(r'\.+', '.', text)
    if text and not text.endswith(('.', '!', '?')):
        text += '.'
    return text

In [137]:
entities = defaultdict(lambda : defaultdict(list))

for k,v in gnostic.items():
    for text in v:
        doc = nlp(clean_text(text))
        for ent in doc.ents:
            if ent.label_ not in ['ORDINAL','CARDINAL','DATE']:
                entities[k][ent.text].append(text)

In [118]:
entities.keys()

dict_keys(['Eugnostos the Blessed', 'The Letter of Peter to Philip', 'The Second Treatise of the Great Seth', 'Authoritative Teaching', 'The Interpretation of Knowledge', 'The Thought of Norea', 'Melchizedek', 'The Apocalypse of Adam', 'The Dialogue of the Savior', 'The Hypostasis of The Archons', 'The Tripartite Tractate', 'The Apocryphon of James', 'second_apocalypse_of_james_verses', 'The Apocryphon of John', 'The Three Steles of Seth', 'The Sentences of Sextus', 'On the Baptism A', 'The Treatise on the Resurrection', 'The (First) Apocalypse of James', 'Allogenes', 'Zostrianos', 'The Testimony of Truth', 'The Sophia of Jesus Christ', 'A Valentinian Exposition', 'The Prayer of the Apostle Paul', 'On the Anointing', 'Marsanes', 'The Thunder Perfect Mind', 'The Gospel of Philip', 'The Concept of Our Great Power', 'Plato Republic 588A-589B', 'Trimorphic Protennoia', 'The Apocalypse of Paul', 'Asclepius 21-29', 'On the Origin of the World', 'Gospel of Thomas', 'The Discourse on the Eight

In [132]:
entities['The Apocalypse of Peter']

defaultdict(list,
            {'Peter': ['As the Savior was sitting in the temple in the three hundredth (year) of the covenant and the agreement of the tenth pillar, and being satisfied with the number of the living, incorruptible Majesty, he said to me, "Peter, blessed are those above belonging to the Father, who revealed life to those who are from the life, through me, since I reminded they who are built on what is strong, that they may hear my word, and distinguish words of unrighteousness and transgression of law from righteousness, as being from the height of every word of this Pleroma of truth, having been enlightened in good pleasure by him whom the principalities sought.',
              'But you yourself, Peter, become perfect in accordance with your name with myself, the one who chose you, because from you I have established a base for the remnant whom I have summoned to knowledge.',
              'And he said to me, "Peter, I have told you many times that they are blind ones

In [146]:
output = []

for book, entity in entities.items():
    for en,text in entity.items():
        if len(text)>1:
            output.append(
                {
                    "book": book,
                    "entity":en,
                    "context":text,
                    "count":len(text)
                }
            )

In [148]:
output[34]

{'book': 'The Apocalypse of Adam',
 'entity': 'Seth',
 'context': ['The revelation which Adam taught his son Seth in the seven hundreth year, saying: Listen to my words, my son Seth.',
  'The revelation which Adam taught his son Seth in the seven hundreth year, saying: Listen to my words, my son Seth.',
  'Now then, my son Seth, I will reveal to you the things which those men whom I saw before me at first revealed to me: after I have completed the times of this generation and the years of the generation have been accomplished, then [...] slave [...].',
  '[...] illuminator [...] will come upon [...] that is dead [...] of Seth.',
  'These are the revelations which Adam made known to Seth, his son, And his son taught his seed about them.',
  'This is the hidden knowledge of Adam, which he gave to Seth, which is the holy baptism of those who know the eternal knowledge through those born of the word and the imperishable illuminators, who came from the holy seed: Yesseus, Mazareus, Yessedek

In [149]:
np.save("enitities.npy",output)

#### Entity Relationship graph

In [150]:
alltext = []

for b,tx in gnostic.items():
    for t in tx:
        ct = clean_text(t)
        if ct:
            alltext.append(ct)

In [171]:
entdata = defaultdict(list)
cooccur = defaultdict(int)

for t in alltext:
    doc = nlp(t)
    ents = [ent.text for ent in doc.ents if ent.label_ not in ['ORDINAL','CARDINAL','DATE']]
    for ent in ents:
        entdata[ent].append(t)
    for i, ent1 in enumerate(ents):
        for ent2 in ents[i+1:]:
            edge = tuple(sorted([ent1,ent2]))
            cooccur[edge]+=1

In [175]:
entities=[]
graphnodes = []
graphedges=[]

for entname,contexts in entdata.items():
    entities.append(
        {
            "entity":entname,
            "context":contexts,
            "count":len(contexts)
        }
    )
    graphnodes.append(
        {
            "id":entname,
            "count":len(contexts)
        }
    )

for (e1,e2),w in cooccur.items():
    graphedges.append({
        "source":e1,
        "target":e2,
        "weight":w
    })

In [176]:
output = {
    "entities":entities,
    "graph":{
        "nodes":graphnodes,
        "edges":graphedges
    }
}

In [177]:
with open("entitygraph.json","w") as f:
    json.dump(output,f)